In [1]:
!pip install transformers datasets
!pip install rouge.score nltk py7zr
!pip install accelerate
!pip install pytorch-accelerated
!pip install transformers==4.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.0 MB/s eta 0:00:0000:01
  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.13.3-py3-none-any.whl.metadata (2.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 5.2 MB/s eta 0:00:0000:0100:01
Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.2.0-py3-none-any.whl (170 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 5.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import torch
import nltk
import numpy as np
import transformers

from datasets import load_dataset, load_metric, load_from_disk

nltk.download('punkt')
torch.mps.set_per_process_memory_fraction(0.0) # Train on Macbook

[nltk_data] Downloading package punkt to /Users/davidlee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
data = load_dataset('samsum')
data.save_to_disk('/Users/davidlee/PycharmProjects/KoodooProject/koodoovoice/dataset_temp/samsum')
data = load_from_disk("/Users/davidlee/PycharmProjects/KoodooProject/koodoovoice/dataset_temp/samsum")

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
metric = load_metric('rouge')
model_checkpoints = 'kabita-choudhary/finetuned-bart-for-conversation-summary'

/var/folders/2l/k2m1kf8908q92grpmz923f600000gn/T/ipykernel_3882/100593241.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')
/Users/davidlee/opt/anaconda3/envs/KoodooProject/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [70]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [71]:
def preprocess_data(data_to_process):
  inputs = [dialogue for dialogue in data_to_process['dialogue']]
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)

  model_inputs['labels'] = targets['input_ids']
  return model_inputs

In [72]:
tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['id', 'dialogue', 'summary'])

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/Users/davidlee/opt/anaconda3/envs/KoodooProject/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [73]:
train_sample = tokenize_data['train'].shuffle(seed=123).select(range(2000))
validation_sample = tokenize_data['validation'].shuffle(seed=123).select(range(800))
test_sample = tokenize_data['test'].shuffle(seed=123).select(range(800))

In [74]:
tokenize_data['train'] = train_sample
tokenize_data['validation'] = validation_sample
tokenize_data['test'] = test_sample

In [82]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [83]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [84]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [85]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=False,
    use_mps_device=True,
    )

In [86]:
trainer = transformers.Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
trainer.train()